In [6]:
#!pip install transformers
#!pip install emoji
#!pip install openpyxl

In [7]:
# Importar las librerías necesarias
import pandas as pd
import emoji
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import Dataset

In [8]:
# Cargar el dataset
df = pd.read_excel('dataset_final.xlsx')
print(f"Datos originales: {len(df)} comentarios")
display(df.head())  # Mostrar los primeros 5 registros

Datos originales: 2814 comentarios


,Comentario,Sentimiento
0,Le están dando eL cartón de mentira para luego...,Neutro
1,ooowww!! Rose Leonor tu amiguis!! la alejitaa ...,Neutro
2,Felicidades primiis ♡♡,Positivo
3,Estamos aun paso para culminar nuestra Ingenie...,Positivo
4,Mijaaaaaaas Requelme Valdivieso,Neutro


In [9]:
# Eliminar filas con comentarios sin etiqueta asignada (etiqueta vacía)
df = df[df['Sentimiento'].notna()]
print(f"Después de eliminar comentarios sin etiqueta: {len(df)} comentarios")
display(df.head())

Después de eliminar comentarios sin etiqueta: 2601 comentarios


,Comentario,Sentimiento
0,Le están dando eL cartón de mentira para luego...,Neutro
1,ooowww!! Rose Leonor tu amiguis!! la alejitaa ...,Neutro
2,Felicidades primiis ♡♡,Positivo
3,Estamos aun paso para culminar nuestra Ingenie...,Positivo
4,Mijaaaaaaas Requelme Valdivieso,Neutro


In [10]:
# Eliminar comentarios duplicados
df = df.drop_duplicates(subset='Comentario')
print(f"Después de eliminar duplicados: {len(df)} comentarios")
display(df.head())

Después de eliminar duplicados: 2501 comentarios


,Comentario,Sentimiento
0,Le están dando eL cartón de mentira para luego...,Neutro
1,ooowww!! Rose Leonor tu amiguis!! la alejitaa ...,Neutro
2,Felicidades primiis ♡♡,Positivo
3,Estamos aun paso para culminar nuestra Ingenie...,Positivo
4,Mijaaaaaaas Requelme Valdivieso,Neutro


In [11]:
# Función para eliminar emojis de los comentarios
def eliminar_emojis(texto):
    return ''.join([char for char in texto if char not in emoji.EMOJI_DATA])

In [12]:
# Aplicar la función de eliminar emojis a la columna 'Comentarios'
df['Comentario'] = df['Comentario'].apply(eliminar_emojis)
print(f"Después de eliminar emojis: {len(df)} comentarios")
display(df.head())


Después de eliminar emojis: 2501 comentarios


,Comentario,Sentimiento
0,Le están dando eL cartón de mentira para luego...,Neutro
1,ooowww!! Rose Leonor tu amiguis!! la alejitaa ...,Neutro
2,Felicidades primiis ♡♡,Positivo
3,Estamos aun paso para culminar nuestra Ingenie...,Positivo
4,Mijaaaaaaas Requelme Valdivieso,Neutro


Mapeo y Codificación de Etiquetas

In [13]:
# Codificar las etiquetas
sentimiento_map = {'Positivo': 1, 'Negativo': 0, 'Neutral': 2}
# Mapear valores y manejar nulos reemplazándolos con un valor predeterminado o eliminándolos
df['Sentimiento_Id'] = df['Sentimiento'].map(sentimiento_map)

# Eliminar filas con valores nulos resultantes del mapeo
df = df[df['Sentimiento_Id'].notna()]

# Convertir a enteros después de manejar los nulos
df['Sentimiento_Id'] = df['Sentimiento_Id'].astype(int)


División de Datos

In [14]:
# Dividir los datos en entrenamiento y prueba
X = df['Comentario'].tolist()
y = df['Sentimiento_Id'].tolist()
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Tokenización

In [15]:
# Tokenización con WordPiece usando el tokenizador de BETO
# Cargar el tokenizador de BETO
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [16]:
# Crear un Dataset compatible con PyTorch
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Asegurar tipo correcto
        return {key: torch.tensor(value) for key, value in item.items()}

Tokenizar los datos

In [17]:
# Tokenizar los datos
def tokenize_data(data, labels):
    encodings = tokenizer(data, truncation=True, padding=True, max_length=128, return_tensors="pt")
    return SentimentDataset(encodings, labels)

train_dataset = tokenize_data(X_train, y_train)
val_dataset = tokenize_data(X_val, y_val)

Configuración del Modelo

In [18]:
# Configurar el modelo
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=3  # Positivo, Negativo, Neutral
)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Entrenamiento del Modelo

In [19]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',         # Carpeta para los resultados
    evaluation_strategy='epoch',   # Evaluar al final de cada época
    learning_rate=2e-5,            # Tasa de aprendizaje
    per_device_train_batch_size=16, # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=16,  # Tamaño del lote de validación
    num_train_epochs=3,            # Número de épocas
    weight_decay=0.01,             # Decaimiento del peso
    logging_dir='./logs',          # Carpeta para los logs
    logging_steps=10,
    save_strategy='epoch',         # Guardar el modelo al final de cada época
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Crear el entrenador

In [20]:
# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


Entrenar el modelo

In [21]:
# Entrenar el modelo
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<ipython-input-16-b3f8f08d22a5>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(value) for key, value in item.items()}


Epoch,Training Loss,Validation Loss
1,0.280500,0.085820
2,0.489200,0.197292
3,0.017600,0.119131


<ipython-input-16-b3f8f08d22a5>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(value) for key, value in item.items()}
<ipython-input-16-b3f8f08d22a5>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(value) for key, value in item.items()}
<ipython-input-16-b3f8f08d22a5>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(value) for key, value in item.items()}


TrainOutput(global_step=270, training_loss=0.1530209183692932, metrics={'train_runtime': 189.2291, 'train_samples_per_second': 22.592, 'train_steps_per_second': 1.427, 'total_flos': 188932906304100.0, 'train_loss': 0.1530209183692932, 'epoch': 3.0})

Evaluación del Modelo

In [23]:
def evaluate_model(trainer, X_test, y_test):
    test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors='pt')
    test_dataset = SentimentDataset(test_encodings, y_test)
    predictions = trainer.predict(test_dataset)
    preds = torch.argmax(torch.tensor(predictions.predictions), dim=1).numpy()
    # Add labels parameter to match the unique classes in y_test
    print(classification_report(y_test, preds, labels=list(set(y_test)), target_names=['Negativo', 'Positivo', 'Neutral']))
    print(confusion_matrix(y_test, preds))

evaluate_model(trainer, X_test, y_test)


<ipython-input-16-b3f8f08d22a5>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(value) for key, value in item.items()}


              precision    recall  f1-score   support

    Negativo       0.97      0.89      0.93        88
    Positivo       0.90      0.98      0.94        91

    accuracy                           0.93       179
   macro avg       0.94      0.93      0.93       179
weighted avg       0.94      0.93      0.93       179

[[78 10]
 [ 2 89]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2687: UserWarning: labels size, 2, does not match size of target_names, 3
  warnings.warn(


Predicción de Nuevos Comentarios

In [26]:
# Función para predecir el sentimiento de un comentario
def predecir_sentimiento(comentario, model, tokenizer):
    # Configurar el dispositivo (GPU si está disponible, de lo contrario CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Mover el modelo al dispositivo

    # Tokenizar el comentario y mover los datos al dispositivo
    inputs = tokenizer(comentario, truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Mover tensores al dispositivo

    # Hacer la predicción con el modelo entrenado
    outputs = model(**inputs)

    # Calcular las probabilidades y predecir la clase
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=1).item()

    # Mapear la predicción a la etiqueta correspondiente
    etiquetas = {0: "Negativo", 1: "Positivo", 2: "Neutral"}
    sentimiento = etiquetas[pred]

    return sentimiento

# Ejemplo de uso
comentario = "No me parece la gran cosa, solo es una graduación más"
resultado = predecir_sentimiento(comentario, model, tokenizer)
print(f"El sentimiento del comentario es: {resultado}")

El sentimiento del comentario es: Negativo
